In [1]:
import numpy as np

In [2]:
list_SD36 = ['6x2pt_LSSTxSO_Y1','6x2pt_LSSTxSO_Y6']
list_COV36 = ['cov_LSSTxSO_Y1_6x2pt.npy','cov_LSSTxSO_Y6_6x2pt.npy']

list_outfiles = ['6x2pt_LSSTxSO_Y1','6x2pt_LSSTxSO_Y6','3x2pt_LSSTxSO_Y1','3x2pt_LSSTxSO_Y6']

dir_datav = "datav/"
dir_bary = "datav/"
dir_COV = "cov/"
dir_LPC = "baryons/"

In [4]:
class gen_LPCi():
    def __init__(self,id_SD,outfile,id_include=None):
        '''
            id_SD = 0~1, int
        '''
        self.id_SD = id_SD
        self.id_include = id_include
        self.outfile = outfile
        self.list_bary_scenario = ['_eagle','_illustris','_TNG100','_mb2','_owls_AGN','_HzAGN']
        self.datav_dmo = self.load_datav(dir_datav,bary_scenario="_dmo",isCutted=1)
        print(self.id_SD)
        print(self.id_include)
        
        self.load_COV()
        print( self.id_SD)
        print( self.datav_dmo.shape, self.COV_cut.shape)
        
        self.cal_invL()
        self.Ndata = len(self.datav_dmo)
        self.Nscenario = len(self.list_bary_scenario)
        
        self.build_Ratio()
        self.build_Delta()
        self.build_DeltaChy()
        self.SVD(in_Delta=self.DeltaChy)
        self.write_LPC()
        
    
    def load_datav(self,dir_path,bary_scenario,isCutted=0):
        fname_datav = dir_path+list_SD36[self.id_SD]+bary_scenario
        print( fname_datav)
        datav = np.genfromtxt(fname_datav,skip_header=0,dtype="int,double",usecols=[0,1],names=["ind","obs"])["obs"]
        if self.id_include is None:
            self.id_include = np.arange(datav.size)
        datav = datav[self.id_include]
        
        self.Ndata_full  = len(datav)
        self.takeout_ID  = np.where(datav > 1e-20)[0]
        self.zero_ID     = np.where(datav < 1e-20)[0]
        
        if isCutted==1:
            return np.array(datav)[self.takeout_ID]
        else:
            return np.array(datav)
        
            
    def load_COV(self):
        
        self.fname_COV = dir_COV+list_COV36[self.id_SD]
        print( self.fname_COV)
        self.COV_full = np.load(self.fname_COV)
        self.COV_full = self.COV_full[self.id_include][:,self.id_include]
        
        self.COV_cut  = self.COV_full[self.takeout_ID][:,self.takeout_ID]
    
    
    def cal_invL(self):
        self.L    = np.linalg.cholesky(self.COV_cut)
        self.invL =  np.linalg.inv(self.L)
        
    def build_Ratio(self):
        self.Ratio     = np.zeros((self.Ndata,self.Nscenario))
        
        for j in range(self.Nscenario):
        
            datav_bary = self.load_datav(dir_bary,bary_scenario=self.list_bary_scenario[j],isCutted=1)
            self.Ratio.T[j] = datav_bary/self.datav_dmo
            
    def build_Delta(self):
        DeltaT = self.Ratio.T*self.datav_dmo-self.datav_dmo
        self.Delta = DeltaT.T
    
    def build_DeltaChy(self):
        self.DeltaChy = np.dot(self.invL,self.Delta)
        
    def SVD(self,in_Delta):
        # self.U stores PC modes
        # PC1 = self.U.T[0]  ; PC2 = self.U.T[1]
        self.U, self.Sdig, VT = np.linalg.svd(in_Delta,full_matrices=True)
    
    def gen_datav_cosmolike_format(self,datav_cut):
        
        datav_full = np.zeros(self.Ndata_full)
        datav_full[self.takeout_ID] = datav_cut
        
        return datav_full

    
    def write_LPC(self):
        
        self.fname_out = dir_LPC+'LPC_'+self.outfile
        
        self.LPC1 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[0]))
        self.LPC2 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[1]))
        self.LPC3 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[2]))
        self.LPC4 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[3]))
        self.LPC5 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[4]))
        self.LPC6 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[5]))
        
        col_names='LPC1 LPC2 LPC3 LPC4 LPC5 LPC6'
        
        LPC_Matrix=np.column_stack((self.LPC1,self.LPC2,self.LPC3,self.LPC4,self.LPC5,self.LPC6))

        np.savetxt(self.fname_out,LPC_Matrix, fmt='%.8e %.8e %.8e %.8e %.8e %.8e')

       

In [5]:
gen_LPCi(0, list_outfiles[0])
gen_LPCi(1, list_outfiles[1])

datav/6x2pt_LSSTxSO_Y1_dmo
0
[   0    1    2 ... 1962 1963 1964]
cov/cov_LSSTxSO_Y1_6x2pt.npy
0
(1583,) (1583, 1583)
datav/6x2pt_LSSTxSO_Y1_eagle
datav/6x2pt_LSSTxSO_Y1_illustris
datav/6x2pt_LSSTxSO_Y1_TNG100
datav/6x2pt_LSSTxSO_Y1_mb2
datav/6x2pt_LSSTxSO_Y1_owls_AGN
datav/6x2pt_LSSTxSO_Y1_HzAGN
datav/6x2pt_LSSTxSO_Y6_dmo
1
[   0    1    2 ... 2067 2068 2069]
cov/cov_LSSTxSO_Y6_6x2pt.npy
1
(1652,) (1652, 1652)
datav/6x2pt_LSSTxSO_Y6_eagle
datav/6x2pt_LSSTxSO_Y6_illustris
datav/6x2pt_LSSTxSO_Y6_TNG100
datav/6x2pt_LSSTxSO_Y6_mb2
datav/6x2pt_LSSTxSO_Y6_owls_AGN
datav/6x2pt_LSSTxSO_Y6_HzAGN


In [6]:
nsource = 10
nshear = nsource*(nsource+1)/2
ntheta = 15
nlens = 10

ndata3x2pt= 1965 - (nlens + nsource + 1)*ntheta
gen_LPCi(0, list_outfiles[2], np.arange(ndata3x2pt))

ndata3x2pt= 2070 - (nlens + nsource + 1)*ntheta
gen_LPCi(1, list_outfiles[3], np.arange(ndata3x2pt))

datav/6x2pt_LSSTxSO_Y1_dmo
0
[   0    1    2 ... 1647 1648 1649]
cov/cov_LSSTxSO_Y1_6x2pt.npy
0
(1323,) (1323, 1323)
datav/6x2pt_LSSTxSO_Y1_eagle
datav/6x2pt_LSSTxSO_Y1_illustris
datav/6x2pt_LSSTxSO_Y1_TNG100
datav/6x2pt_LSSTxSO_Y1_mb2
datav/6x2pt_LSSTxSO_Y1_owls_AGN
datav/6x2pt_LSSTxSO_Y1_HzAGN
datav/6x2pt_LSSTxSO_Y6_dmo
1
[   0    1    2 ... 1752 1753 1754]
cov/cov_LSSTxSO_Y6_6x2pt.npy
1
(1392,) (1392, 1392)
datav/6x2pt_LSSTxSO_Y6_eagle
datav/6x2pt_LSSTxSO_Y6_illustris
datav/6x2pt_LSSTxSO_Y6_TNG100
datav/6x2pt_LSSTxSO_Y6_mb2
datav/6x2pt_LSSTxSO_Y6_owls_AGN
datav/6x2pt_LSSTxSO_Y6_HzAGN
